# Collate demographic data for each LSOA

This notebook collates various and saves as `collated_data.csv` in data folder. 

In [1]:
import numpy as np
import pandas as pd

## Unzip required travel matrix file

In [2]:
import zipfile
with zipfile.ZipFile("./data/lsoa_travel_time_matrix_calibrated.zip","r") as zip_ref:
    zip_ref.extractall("./data/")

## Import admissions data

In [3]:
data = pd.read_csv('./data/admissions_2017-2019.csv', index_col='area')
data.rename(columns={'Admissions': 'admissions'}, inplace=True)

data.head(2)

,admissions
area,
Welwyn Hatfield 010F,0.666667
Welwyn Hatfield 012A,4.000000


## Import units

In [4]:
units = pd.read_csv('./data/stroke_hospitals_2022.csv')

units.head(2)

,Index,Postcode,Hospital_name,Use,Country,Strategic Clinical Network,Health Board / Trust,Stroke Team,SSNAP name,Admissions 21/22,Thrombolysis,ivt_rate,Easting,Northing,Neuroscience,30 England Thrombectomy Example,Notes
0,1,RM70AG,RM70AG,1,England,London SCN,Barking; Havering and Redbridge University Hos...,"Queen's Hospital, Romford",Queens Hospital Romford HASU,981,117,11.9,551118,187780,1.0,NaN,NaN
1,2,E11BB,E11BB,1,England,London SCN,Barts Health NHS Trust,The Royal London Hospital,Royal London Hospital HASU,861,115,13.4,534829,181798,1.0,1.0,NaN


Get postcodes of unit in use for ivt)

In [5]:
mask = units['Use'] == 1
units_in_use = list(units[mask]['Postcode'])
units_in_use[0:5]

['RM70AG', 'E11BB', 'SW66SX', 'SE59RW', 'BR68ND']

Get postcodes of units providing thrombectomy.

In [6]:
mask = units['Neuroscience'] == 1
thromectomy_units = list(units[mask]['Postcode'])
thromectomy_units[0:5]

['RM70AG', 'E11BB', 'SW66SX', 'SE59RW', 'SW170QT']

## Import travel time to stroke units

In [7]:
travel_time = pd.read_csv('./data/lsoa_travel_time_matrix_calibrated.csv', index_col='LSOA')
# Limit to units in use
travel_time = travel_time[units_in_use]
travel_time.head(2)

,RM70AG,E11BB,SW66SX,SE59RW,BR68ND,HA13UJ,SW170QT,NW12BU,DE223NE,NN15BD,...,LL137TD,LL572PW,CF144XW,CF479DT,CF311RQ,SY231ER,SA148QF,SA312AF,SA612PZ,SA66NL
LSOA,,,,,,,,,,,,,,,,,,,,,
City of London 001A,34.3,11.7,26.7,18.1,39.6,32.1,27.8,13.8,142.9,82.6,...,202.0,276.2,165.4,190.2,180.5,256.8,210.6,227.8,259.0,203.1
City of London 001B,33.2,10.6,28.9,18.1,38.6,33.2,26.7,13.8,142.9,82.6,...,203.1,276.2,166.5,191.2,181.6,256.8,212.7,228.9,261.1,204.1


Get closest unit and time  for IVT (convert series to dataframes)

In [8]:
closest_unit = pd.DataFrame(travel_time.idxmin(axis=1), columns=['closest_ivt_unit'])

In [9]:
closest_unit_time = pd.DataFrame(travel_time.min(axis=1), columns=['closest_ivt_time'])

Merge into admissions

In [10]:
data = data.merge(closest_unit, left_index=True, right_index=True, how='left')
data = data.merge(closest_unit_time, left_index=True, right_index=True,  how='left')
data.head(2)

,admissions,closest_ivt_unit,closest_ivt_time
area,,,
Welwyn Hatfield 010F,0.666667,SG14AB,19.2
Welwyn Hatfield 012A,4.000000,SG14AB,20.3


Add travel times to thrombectomy units 

In [11]:
mask = units['Neuroscience'] == 1
thromectomy_units = list(units[mask]['Postcode'])

In [12]:
closest_thrombectomy_unit = pd.DataFrame(travel_time[thromectomy_units].idxmin(axis=1), columns=['closest_mt_unit'])
closest_thrombectomy_time = pd.DataFrame(travel_time[thromectomy_units].min(axis=1), columns=['closest_mt_time'])

In [13]:
data = data.merge(closest_thrombectomy_unit, left_index=True, right_index=True, how='left')
data = data.merge(closest_thrombectomy_time, left_index=True, right_index=True, how='left')
data.head(2)

,admissions,closest_ivt_unit,closest_ivt_time,closest_mt_unit,closest_mt_time
area,,,,,
Welwyn Hatfield 010F,0.666667,SG14AB,19.2,NW12BU,37.5
Welwyn Hatfield 012A,4.000000,SG14AB,20.3,NW12BU,37.5


Add travel time between IVT and ET units.

In [14]:
inter_hospital_times = pd.read_csv('./data/inter_hospital_time_calibrated.csv', index_col='from_postcode')

In [15]:
transfer_times = []
for index, value in data.iterrows():
    ivt_unit = value['closest_ivt_unit']
    mt_unit = value['closest_mt_unit']
    transfer_time = inter_hospital_times.loc[ivt_unit][mt_unit]
    transfer_times.append(transfer_time)

In [16]:
data['mt_transfer_time'] = transfer_times
data.head(2)

,admissions,closest_ivt_unit,closest_ivt_time,closest_mt_unit,closest_mt_time,mt_transfer_time
area,,,,,,
Welwyn Hatfield 010F,0.666667,SG14AB,19.2,NW12BU,37.5,47.2
Welwyn Hatfield 012A,4.000000,SG14AB,20.3,NW12BU,37.5,47.2


In [17]:
lsoa_index = data.index
lsoa_index

Index(['Welwyn Hatfield 010F', 'Welwyn Hatfield 012A', 'Welwyn Hatfield 002F',
       'Welwyn Hatfield 002E', 'Welwyn Hatfield 010A', 'Welwyn Hatfield 010B',
       'Welwyn Hatfield 012B', 'Welwyn Hatfield 011D', 'Welwyn Hatfield 011A',
       'Welwyn Hatfield 012C',
       ...
       'Selby 008A', 'Selby 007C', 'Selby 008B', 'Selby 007E', 'Selby 006D',
       'Selby 009C', 'Selby 007D', 'Selby 007B', 'Selby 006A', 'Selby 007A'],
      dtype='object', name='area', length=32843)

## Add thrombolysis rate of closest IVT unit

In [18]:
mask = units['Use'] == 1
ivt_rate = units[mask][['Postcode', 'ivt_rate']]

data = data.merge(
    ivt_rate, left_on='closest_ivt_unit', right_on='Postcode', how='left')
data.drop('Postcode', axis=1, inplace=True)
data.set_index(lsoa_index, inplace=True)

In [19]:
data.head(2)

,admissions,closest_ivt_unit,closest_ivt_time,closest_mt_unit,closest_mt_time,mt_transfer_time,ivt_rate
area,,,,,,,
Welwyn Hatfield 010F,0.666667,SG14AB,19.2,NW12BU,37.5,47.2,6.8
Welwyn Hatfield 012A,4.000000,SG14AB,20.3,NW12BU,37.5,47.2,6.8


## Import deprivation demographics

In [20]:
deprivation = pd.read_csv(
    './data/demographic/File_7_-_All_IoD2019_Scores__Ranks__Deciles_and_Population_Denominators_3.csv',
    index_col='LSOA name (2011)')
cols_to_drop = ['LSOA code (2011)','Local Authority District code (2019)']
deprivation.drop(cols_to_drop, axis=1, inplace=True)

In [21]:
deprivation.head(2)

,Local Authority District name (2019),Index of Multiple Deprivation (IMD) Score,Index of Multiple Deprivation (IMD) Rank (where 1 is most deprived),Index of Multiple Deprivation (IMD) Decile (where 1 is most deprived 10% of LSOAs),Income Score (rate),Income Rank (where 1 is most deprived),Income Decile (where 1 is most deprived 10% of LSOAs),Employment Score (rate),Employment Rank (where 1 is most deprived),Employment Decile (where 1 is most deprived 10% of LSOAs),...,Indoors Sub-domain Rank (where 1 is most deprived),Indoors Sub-domain Decile (where 1 is most deprived 10% of LSOAs),Outdoors Sub-domain Score,Outdoors Sub-domain Rank (where 1 is most deprived),Outdoors Sub-domain Decile (where 1 is most deprived 10% of LSOAs),Total population: mid 2015 (excluding prisoners),Dependent Children aged 0-15: mid 2015 (excluding prisoners),Population aged 16-59: mid 2015 (excluding prisoners),Older population aged 60 and over: mid 2015 (excluding prisoners),Working age population 18-59/64: for use with Employment Deprivation Domain (excluding prisoners)
LSOA name (2011),,,,,,,,,,,,,,,,,,,,,
City of London 001A,City of London,6.208,29199,9,0.007,32831,10,0.010,32742,10,...,16364,5,1.503,1615,1,1296,175,656,465,715
City of London 001B,City of London,5.143,30379,10,0.034,29901,10,0.027,31190,10,...,22676,7,1.196,2969,1,1156,182,580,394,620


In [22]:
data = data.merge(deprivation, left_index=True, right_index=True,  how='left')

data.set_index(lsoa_index, inplace=True)

data.head(2)

,admissions,closest_ivt_unit,closest_ivt_time,closest_mt_unit,closest_mt_time,mt_transfer_time,ivt_rate,Local Authority District name (2019),Index of Multiple Deprivation (IMD) Score,Index of Multiple Deprivation (IMD) Rank (where 1 is most deprived),...,Indoors Sub-domain Rank (where 1 is most deprived),Indoors Sub-domain Decile (where 1 is most deprived 10% of LSOAs),Outdoors Sub-domain Score,Outdoors Sub-domain Rank (where 1 is most deprived),Outdoors Sub-domain Decile (where 1 is most deprived 10% of LSOAs),Total population: mid 2015 (excluding prisoners),Dependent Children aged 0-15: mid 2015 (excluding prisoners),Population aged 16-59: mid 2015 (excluding prisoners),Older population aged 60 and over: mid 2015 (excluding prisoners),Working age population 18-59/64: for use with Employment Deprivation Domain (excluding prisoners)
area,,,,,,,,,,,,,,,,,,,,,
Welwyn Hatfield 010F,0.666667,SG14AB,19.2,NW12BU,37.5,47.2,6.8,Welwyn Hatfield,15.616,18334,...,25594,8,0.799,5658,2,1584,322,975,287,1004
Welwyn Hatfield 012A,4.000000,SG14AB,20.3,NW12BU,37.5,47.2,6.8,Welwyn Hatfield,33.313,6540,...,19904,7,0.484,8574,3,1822,283,1242,297,1253


## Import ethnicity

In [23]:
ethnicity = pd.read_csv(
    './data/demographic/ethnicity.csv',
    index_col='geography')

# Include only ethnicity for all people (no age/gender breakdown)
cols_to_include = [
    x for x in list(ethnicity) if 'Sex: All persons; Age: All categories' in x]

# Get selected data
extract = ethnicity[cols_to_include]

# Truncate field name
rename_dict = dict()
for field in list(extract):
    rename_dict[field] = field[44:]
extract = extract.rename(rename_dict, axis='columns')
    
# Merge in with data
data = data.merge(extract, left_index=True, right_index=True,  how='left')

data.set_index(lsoa_index, inplace=True)

data.head(2)

,admissions,closest_ivt_unit,closest_ivt_time,closest_mt_unit,closest_mt_time,mt_transfer_time,ivt_rate,Local Authority District name (2019),Index of Multiple Deprivation (IMD) Score,Index of Multiple Deprivation (IMD) Rank (where 1 is most deprived),...,Ethnic Group: Asian/Asian British: Bangladeshi; measures: Value,Ethnic Group: Asian/Asian British: Chinese; measures: Value,Ethnic Group: Asian/Asian British: Other Asian; measures: Value,Ethnic Group: Black/African/Caribbean/Black British: Total; measures: Value,Ethnic Group: Black/African/Caribbean/Black British: African; measures: Value,Ethnic Group: Black/African/Caribbean/Black British: Caribbean; measures: Value,Ethnic Group: Black/African/Caribbean/Black British: Other Black; measures: Value,Ethnic Group: Other ethnic group: Total; measures: Value,Ethnic Group: Other ethnic group: Arab; measures: Value,Ethnic Group: Other ethnic group: Any other ethnic group; measures: Value
area,,,,,,,,,,,,,,,,,,,,,
Welwyn Hatfield 010F,0.666667,SG14AB,19.2,NW12BU,37.5,47.2,6.8,Welwyn Hatfield,15.616,18334,...,14,23,35,105,73,30,2,14,6,8
Welwyn Hatfield 012A,4.000000,SG14AB,20.3,NW12BU,37.5,47.2,6.8,Welwyn Hatfield,33.313,6540,...,28,37,23,151,127,11,13,25,11,14


## Import health

In [24]:
health = pd.read_csv(
    './data/demographic/general_health.csv',
    index_col='geography')

# Include only ethnicity for all people (no age/gender breakdown)
cols_to_include = [
    x for x in list(health) if 'Sex: All persons; Age: All categories' in x]

# Get selected data
extract = health[cols_to_include]

# Truncate field name
rename_dict = dict()
for field in list(extract):
    rename_dict[field] = field[44:]
extract = extract.rename(rename_dict, axis='columns')
    
# Merge in with data
data = data.merge(extract, left_index=True, right_index=True,  how='left')
data.set_index(lsoa_index, inplace=True)

data.head(2)

,admissions,closest_ivt_unit,closest_ivt_time,closest_mt_unit,closest_mt_time,mt_transfer_time,ivt_rate,Local Authority District name (2019),Index of Multiple Deprivation (IMD) Score,Index of Multiple Deprivation (IMD) Rank (where 1 is most deprived),...,General Health: Fair health; Ethnic Group: Other ethnic group; measures: Value,General Health: Bad or very bad health; Ethnic Group: All categories: Ethnic group; measures: Value,General Health: Bad or very bad health; Ethnic Group: White: Total; measures: Value,General Health: Bad or very bad health; Ethnic Group: White: English/Welsh/Scottish/Northern Irish/British; measures: Value,General Health: Bad or very bad health; Ethnic Group: White: Irish; measures: Value,General Health: Bad or very bad health; Ethnic Group: White: Other White; measures: Value,General Health: Bad or very bad health; Ethnic Group: Mixed/multiple ethnic group; measures: Value,General Health: Bad or very bad health; Ethnic Group: Asian/Asian British; measures: Value,General Health: Bad or very bad health; Ethnic Group: Black/African/Caribbean/Black British; measures: Value,General Health: Bad or very bad health; Ethnic Group: Other ethnic group; measures: Value
area,,,,,,,,,,,,,,,,,,,,,
Welwyn Hatfield 010F,0.666667,SG14AB,19.2,NW12BU,37.5,47.2,6.8,Welwyn Hatfield,15.616,18334,...,1,64,55,51,1,3,3,2,4,0
Welwyn Hatfield 012A,4.000000,SG14AB,20.3,NW12BU,37.5,47.2,6.8,Welwyn Hatfield,33.313,6540,...,1,103,98,87,5,6,1,1,2,1


## Import rural vs urban

In [25]:
rural_urban = pd.read_csv(
    './data/demographic/Rural_Urban_Classification_2011_of_Lower_Layer_Super_Output_Areas_in_England_and_Wales.csv',
    index_col='LSOA11NM')
cols_to_drop = ['LSOA11CD','RUC11CD', 'FID']
rural_urban.drop(cols_to_drop, axis=1, inplace=True)

data = data.merge(rural_urban, left_index=True, right_index=True,  how='left')
data.set_index(lsoa_index, inplace=True)

data.head(2)

,admissions,closest_ivt_unit,closest_ivt_time,closest_mt_unit,closest_mt_time,mt_transfer_time,ivt_rate,Local Authority District name (2019),Index of Multiple Deprivation (IMD) Score,Index of Multiple Deprivation (IMD) Rank (where 1 is most deprived),...,General Health: Bad or very bad health; Ethnic Group: All categories: Ethnic group; measures: Value,General Health: Bad or very bad health; Ethnic Group: White: Total; measures: Value,General Health: Bad or very bad health; Ethnic Group: White: English/Welsh/Scottish/Northern Irish/British; measures: Value,General Health: Bad or very bad health; Ethnic Group: White: Irish; measures: Value,General Health: Bad or very bad health; Ethnic Group: White: Other White; measures: Value,General Health: Bad or very bad health; Ethnic Group: Mixed/multiple ethnic group; measures: Value,General Health: Bad or very bad health; Ethnic Group: Asian/Asian British; measures: Value,General Health: Bad or very bad health; Ethnic Group: Black/African/Caribbean/Black British; measures: Value,General Health: Bad or very bad health; Ethnic Group: Other ethnic group; measures: Value,RUC11
area,,,,,,,,,,,,,,,,,,,,,
Welwyn Hatfield 010F,0.666667,SG14AB,19.2,NW12BU,37.5,47.2,6.8,Welwyn Hatfield,15.616,18334,...,64,55,51,1,3,3,2,4,0,Urban city and town
Welwyn Hatfield 012A,4.000000,SG14AB,20.3,NW12BU,37.5,47.2,6.8,Welwyn Hatfield,33.313,6540,...,103,98,87,5,6,1,1,2,1,Urban city and town


## Import age demographics

In [26]:
ages = pd.read_csv(
    './data/demographic/mid_2018_persons.csv',
    index_col='LSOA')

all_ages = ages['All Ages']
ages.drop('All Ages', axis=1, inplace=True)
data['All persons'] = all_ages

# Change '90+' to 91
rename_dict = dict()
rename_dict['90+'] = '91'
ages = ages.rename(rename_dict, axis='columns')

age_bands = pd.DataFrame()

for band in np.arange(0, 96,5):
    cols_to_get = []
    for field in list(ages):
        if int(int(field)/5) * 5 == band:
            cols_to_get.append(field)
    extract = ages[cols_to_get]
    age_bands[f'age band {band}'] = extract.sum(axis=1)
    
data = data.merge(age_bands, left_index=True, right_index=True,  how='left')
data.set_index(lsoa_index, inplace=True)
data.head(2)

,admissions,closest_ivt_unit,closest_ivt_time,closest_mt_unit,closest_mt_time,mt_transfer_time,ivt_rate,Local Authority District name (2019),Index of Multiple Deprivation (IMD) Score,Index of Multiple Deprivation (IMD) Rank (where 1 is most deprived),...,age band 50,age band 55,age band 60,age band 65,age band 70,age band 75,age band 80,age band 85,age band 90,age band 95
area,,,,,,,,,,,,,,,,,,,,,
Welwyn Hatfield 010F,0.666667,SG14AB,19.2,NW12BU,37.5,47.2,6.8,Welwyn Hatfield,15.616,18334,...,106,109,92,64,62,48,20,9,7,0.0
Welwyn Hatfield 012A,4.000000,SG14AB,20.3,NW12BU,37.5,47.2,6.8,Welwyn Hatfield,33.313,6540,...,122,105,79,51,55,41,30,26,19,0.0


In [27]:
ages = pd.read_csv(
    './data/demographic/mid_2018_females.csv',
    index_col='LSOA')

all_ages = ages['All Ages']
ages.drop('All Ages', axis=1, inplace=True)
data['All females'] = all_ages

# Change '90+' to 91
rename_dict = dict()
rename_dict['90+'] = '91'
ages = ages.rename(rename_dict, axis='columns')

age_bands = pd.DataFrame()

for band in np.arange(0, 96,5):
    cols_to_get = []
    for field in list(ages):
        if int(int(field)/5) * 5 == band:
            cols_to_get.append(field)
    extract = ages[cols_to_get]
    age_bands[f'age band females {band}'] = extract.sum(axis=1)
    
data = data.merge(age_bands, left_index=True, right_index=True,  how='left')
data.set_index(lsoa_index, inplace=True)
data.head(2)

,admissions,closest_ivt_unit,closest_ivt_time,closest_mt_unit,closest_mt_time,mt_transfer_time,ivt_rate,Local Authority District name (2019),Index of Multiple Deprivation (IMD) Score,Index of Multiple Deprivation (IMD) Rank (where 1 is most deprived),...,age band females 50,age band females 55,age band females 60,age band females 65,age band females 70,age band females 75,age band females 80,age band females 85,age band females 90,age band females 95
area,,,,,,,,,,,,,,,,,,,,,
Welwyn Hatfield 010F,0.666667,SG14AB,19.2,NW12BU,37.5,47.2,6.8,Welwyn Hatfield,15.616,18334,...,53,47,55,32,35,18,5,5,4,0.0
Welwyn Hatfield 012A,4.000000,SG14AB,20.3,NW12BU,37.5,47.2,6.8,Welwyn Hatfield,33.313,6540,...,64,42,36,15,30,27,15,17,12,0.0


In [28]:
ages = pd.read_csv(
    './data/demographic/mid_2018_males.csv',
    index_col='LSOA')

all_ages = ages['All Ages']
ages.drop('All Ages', axis=1, inplace=True)
data['All males'] = all_ages

# Change '90+' to 91
rename_dict = dict()
rename_dict['90+'] = '91'
ages = ages.rename(rename_dict, axis='columns')

age_bands = pd.DataFrame()

for band in np.arange(0, 96,5):
    cols_to_get = []
    for field in list(ages):
        if int(int(field)/5) * 5 == band:
            cols_to_get.append(field)
    extract = ages[cols_to_get]
    age_bands[f'age band males {band}'] = extract.sum(axis=1)
    
data = data.merge(age_bands, left_index=True, right_index=True,  how='left')
data.set_index(lsoa_index, inplace=True)
data.head(2)

,admissions,closest_ivt_unit,closest_ivt_time,closest_mt_unit,closest_mt_time,mt_transfer_time,ivt_rate,Local Authority District name (2019),Index of Multiple Deprivation (IMD) Score,Index of Multiple Deprivation (IMD) Rank (where 1 is most deprived),...,age band males 50,age band males 55,age band males 60,age band males 65,age band males 70,age band males 75,age band males 80,age band males 85,age band males 90,age band males 95
area,,,,,,,,,,,,,,,,,,,,,
Welwyn Hatfield 010F,0.666667,SG14AB,19.2,NW12BU,37.5,47.2,6.8,Welwyn Hatfield,15.616,18334,...,53,62,37,32,27,30,15,4,3,0.0
Welwyn Hatfield 012A,4.000000,SG14AB,20.3,NW12BU,37.5,47.2,6.8,Welwyn Hatfield,33.313,6540,...,58,63,43,36,25,14,15,9,7,0.0


## Save output

In [29]:
data.to_csv('./data/collated_data.csv', index_label='LSOA')